# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 1:48PM,258272,16,9212677,Sartorius Bioprocess Solutions,Released
1,Feb 27 2023 1:48PM,258271,16,9212658,Sartorius Bioprocess Solutions,Released
2,Feb 27 2023 1:45PM,258270,16,9212653,Sartorius Bioprocess Solutions,Released
3,Feb 27 2023 1:36PM,258269,12,HH9028,Hush Hush,Released
4,Feb 27 2023 1:30PM,258266,19,60021512,"GCH Granules USA, Inc.",Released
5,Feb 27 2023 1:30PM,258266,19,60021497,"GCH Granules USA, Inc.",Released
6,Feb 27 2023 1:27PM,258264,19,9014377-BO,Eywa Pharma Inc.,Released
7,Feb 27 2023 1:27PM,258264,19,9014375-BO1,Eywa Pharma Inc.,Released
8,Feb 27 2023 1:27PM,258264,19,8749039-BO,Eywa Pharma Inc.,Released
9,Feb 27 2023 1:27PM,258264,19,8749035-BO1,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,258266,Released,2
47,258269,Released,1
48,258270,Released,1
49,258271,Released,1
50,258272,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258266,NaN,NaN,2.0
258269,NaN,NaN,1.0
258270,NaN,NaN,1.0
258271,NaN,NaN,1.0
258272,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258149,0.0,0.0,18.0
258153,0.0,30.0,1.0
258162,0.0,0.0,58.0
258166,8.0,9.0,0.0
258188,0.0,0.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258149,0,0,18
258153,0,30,1
258162,0,0,58
258166,8,9,0
258188,0,0,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258149,0,0,18
1,258153,0,30,1
2,258162,0,0,58
3,258166,8,9,0
4,258188,0,0,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258149,,,18
1,258153,,30,1
2,258162,,,58
3,258166,8,9,
4,258188,,,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions
3,Feb 27 2023 1:36PM,258269,12,Hush Hush
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc."
6,Feb 27 2023 1:27PM,258264,19,Eywa Pharma Inc.
11,Feb 27 2023 1:26PM,258265,19,"GCH Granules USA, Inc."
16,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC"
17,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc."
22,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,,,1
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions,,,1
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions,,,1
3,Feb 27 2023 1:36PM,258269,12,Hush Hush,,,1
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",,,2
5,Feb 27 2023 1:27PM,258264,19,Eywa Pharma Inc.,,,5
6,Feb 27 2023 1:26PM,258265,19,"GCH Granules USA, Inc.",,,5
7,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",,,1
8,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",,,5
9,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,1,,
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions,1,,
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions,1,,
3,Feb 27 2023 1:36PM,258269,12,Hush Hush,1,,
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",2,,
5,Feb 27 2023 1:27PM,258264,19,Eywa Pharma Inc.,5,,
6,Feb 27 2023 1:26PM,258265,19,"GCH Granules USA, Inc.",5,,
7,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",1,,
8,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",5,,
9,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,1,,
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions,1,,
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions,1,,
3,Feb 27 2023 1:36PM,258269,12,Hush Hush,1,,
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Feb 27 2023 1:36PM,258269,12,Hush Hush,1.0,NaN,NaN
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:48PM,258272,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Feb 27 2023 1:48PM,258271,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Feb 27 2023 1:45PM,258270,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Feb 27 2023 1:36PM,258269,12,Hush Hush,1.0,0.0,0.0
4,Feb 27 2023 1:30PM,258266,19,"GCH Granules USA, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4389483,328.0,43.0,0.0
12,516514,1.0,1.0,0.0
15,774611,37.0,71.0,36.0
16,1033014,4.0,0.0,0.0
19,3873607,38.0,20.0,8.0
21,258234,1.0,0.0,0.0
22,516510,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4389483,328.0,43.0,0.0
1,12,516514,1.0,1.0,0.0
2,15,774611,37.0,71.0,36.0
3,16,1033014,4.0,0.0,0.0
4,19,3873607,38.0,20.0,8.0
5,21,258234,1.0,0.0,0.0
6,22,516510,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,328.0,43.0,0.0
1,12,1.0,1.0,0.0
2,15,37.0,71.0,36.0
3,16,4.0,0.0,0.0
4,19,38.0,20.0,8.0
5,21,1.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,328.0
1,12,Released,1.0
2,15,Released,37.0
3,16,Released,4.0
4,19,Released,38.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,36.0,0.0,8.0,0.0,0.0
Executing,43.0,1.0,71.0,0.0,20.0,0.0,0.0
Released,328.0,1.0,37.0,4.0,38.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,36.0,0.0,8.0,0.0,0.0
1,Executing,43.0,1.0,71.0,0.0,20.0,0.0,0.0
2,Released,328.0,1.0,37.0,4.0,38.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,36.0,0.0,8.0,0.0,0.0
1,Executing,43.0,1.0,71.0,0.0,20.0,0.0,0.0
2,Released,328.0,1.0,37.0,4.0,38.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()